In [1]:
!pip install rasterio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 115.4 MB/s eta 0:00:00


In [2]:
!pip install sahi --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 2.2 MB/s eta 0:00:00


In [7]:
!pip install sahi ultralytics rasterio geopandas shapely opencv-python-headless --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.9 MB/s eta 0:00:00


In [4]:
import os
import sys
import cv2
import torch
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Importaciones de SAHI
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
from sahi.utils.cv import visualize_object_predictions

In [17]:


# ==========================================
# ⚙️ CONFIGURATION
# ==========================================
# Asegúrate de que este archivo exista
IMAGE_PATH = "Sample-drone-data-copy.tif"
MODEL_PATH = "yolo_best.pt"
OUTPUT_DIR = "final_results"

# INFERENCE SETTINGS
SLICE_SIZE = 640         # Must match your training size
OVERLAP_RATIO = 0.2      # 20% overlap
CONFIDENCE = 0.25        # 25% minimum confidence
IOU_THRESHOLD = 0.5      # 50% overlap threshold (NMS)
# SLICE_SIZE = 640        # Tamaño del tile (estándar para YOLOv8)
# OVERLAP_RATIO = 0.2     # 20% de solapamiento
# CONFIDENCE = 0.65       # Umbral de confianza
# IOU_THRESHOLD = 0.3     # NMS para eliminar duplicados

# VISUALIZATION SETTINGS
DOT_SIZE = 10
DOT_COLOR = (0, 0, 255) # Rojo (BGR)
TEXT_COLOR = (255, 255, 255)
BG_COLOR = (0, 0, 0)

# ==========================================
# 🚀 SMART DEVICE DETECTION
# ==========================================
def get_optimal_device():
    if torch.cuda.is_available():
        print("✅ Device: CUDA (GPU)")
        return 'cuda:0'
    elif torch.backends.mps.is_available():
        print("✅ Device: MPS (Mac M1/M2)")
        return 'mps'
    else:
        print("⚠️ Device: CPU (Lento)")
        return 'cpu'

# ==========================================
# 🛠️ MAIN EXECUTION
# ==========================================
def main():
    # 0. Validaciones previas
    if not os.path.exists(IMAGE_PATH):
        sys.exit(f"❌ Error: No se encuentra la imagen: {IMAGE_PATH}")
    if not os.path.exists(MODEL_PATH):
        sys.exit(f"❌ Error: No se encuentra el modelo: {MODEL_PATH}")

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    device = get_optimal_device()

    # --------------------------------------
    # 1. CARGAR RASTERIO, PREPARAR ARRAY Y CONVERTIR 16-BIT A 8-BIT
    # --------------------------------------
    print("⏳ Leyendo GeoTIFF con Rasterio para asegurar bandas 2, 3, 1 y escalando a 8-bit...")
    with rasterio.open(IMAGE_PATH) as src:
        transform = src.transform
        crs = src.crs

        # Leemos bandas 2, 3, y 1 en ese orden
        # Formato inicial: (3, H, W)
        img_array = src.read([2, 3, 1])

        # FIX CRUCIAL: Escalar de 16-bit (uint16) a 8-bit (uint8) para compatibilidad con YOLO/CV
        # Si ya es uint8, esto no hará nada. Si es uint16, lo normaliza.
        max_val_in_data = np.max(img_array)
        if max_val_in_data > 255:
            # Escalar y convertir a float temporalmente
            img_scaled = img_array.astype(np.float32) * (255.0 / max_val_in_data)
            # Convertir al tipo final (uint8)
            img_final_uint8 = img_scaled.astype(np.uint8)
        else:
            # Si ya está en rango 0-255, solo convertimos a uint8
            img_final_uint8 = img_array.astype(np.uint8)

        # Transponer para YOLO/SAHI: (Alto, Ancho, Canales) -> (H, W, 3)
        image_for_sahi = np.transpose(img_final_uint8, (1, 2, 0))

        # image_cv (BGR) es para dibujar con OpenCV
        image_cv = cv2.cvtColor(image_for_sahi, cv2.COLOR_RGB2BGR)

    # --------------------------------------
    # 2. CARGAR MODELO (SAHI WRAPPER)
    # --------------------------------------
    print(f"⏳ Cargando modelo YOLOv8 desde {MODEL_PATH}...")
    try:
        detection_model = AutoDetectionModel.from_pretrained(
            model_type='yolov8',
            model_path=MODEL_PATH,
            confidence_threshold=CONFIDENCE,
            device=device
        )
    except Exception as e:
        # Si falla aquí, el modelo .pt está dañado.
        sys.exit(f"❌ Error cargando modelo SAHI (Revisa el archivo .pt): {e}")

    # --------------------------------------
    # 3. INFERENCIA CON SAHI (SLICING)
    # --------------------------------------
    print("⏳ Iniciando inferencia por Tiles con array de píxeles...")

    # Pasamos el array de NumPy, NO la ruta, para evitar el error de lectura de TIFF.
    result = get_sliced_prediction(
        image_for_sahi, # <--- Array 8-bit listo para YOLO
        detection_model,
        slice_height=SLICE_SIZE,
        slice_width=SLICE_SIZE,
        overlap_height_ratio=OVERLAP_RATIO,
        overlap_width_ratio=OVERLAP_RATIO,
        postprocess_type="NMS",
        postprocess_match_metric="IOU",
        postprocess_match_threshold=IOU_THRESHOLD,
        verbose=1
    )

    total_trees = len(result.object_prediction_list)
    print(f"✅ Inferencia completada. Palmas detectadas: {total_trees}")

    # --------------------------------------
    # 4. DIBUJAR CENTROIDES (VISUAL)
    # --------------------------------------
    # ... (El código de dibujo)
    for pred in result.object_prediction_list:
        bbox = pred.bbox
        center_x = int((bbox.minx + bbox.maxx) / 2)
        center_y = int((bbox.miny + bbox.maxy) / 2)
        cv2.circle(image_cv, (center_x, center_y), DOT_SIZE, DOT_COLOR, -1)

    # Etiqueta tipo Dashboard
    label_text = f" Palms: {total_trees} "
    cv2.rectangle(image_cv, (0, 0), (350, 60), BG_COLOR, -1)
    cv2.putText(image_cv, label_text, (10, 45),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, TEXT_COLOR, 3)

    # Guardar PNG
    out_png = os.path.join(OUTPUT_DIR, "visual_map_centroids.png")
    cv2.imwrite(out_png, image_cv)
    print(f"💾 Imagen visual (PNG) guardada en: {out_png}")

    # --------------------------------------
    # 5. EXPORTAR GEOJSON (GEO-REFERENCIADO)
    # --------------------------------------
    records = []

    for i, pred in enumerate(result.object_prediction_list):
        bbox = pred.bbox
        score = pred.score.value
        px = (bbox.minx + bbox.maxx) / 2
        py = (bbox.miny + bbox.maxy) / 2

        # Transformación Affine: Pixel (col, row) -> Coordenada Real (X, Y)
        real_x, real_y = rasterio.transform.xy(transform, py, px, offset='center')

        records.append({
            "id": i,
            "confidence": round(score, 4),
            "pixel_x": int(px),
            "pixel_y": int(py),
            "geometry": Point(real_x, real_y)
        })

    # Crear y guardar GeoDataFrame
    if records:
        gdf = gpd.GeoDataFrame(records, crs=crs)
        out_geojson = os.path.join(OUTPUT_DIR, "detected_palms.geojson")
        gdf.to_file(out_geojson, driver="GeoJSON")
        print(f"💾 GeoJSON guardado en: {out_geojson}")

        out_csv = os.path.join(OUTPUT_DIR, "detected_palms.csv")
        pd.DataFrame(records).drop(columns='geometry').to_csv(out_csv, index=False)

    print("\n🚀 PROCESO FINALIZADO CON ÉXITO")

# --------------------------------------
# ENTRY POINT
# --------------------------------------
if __name__ == "__main__":
    main()

✅ Device: CUDA (GPU)
⏳ Leyendo GeoTIFF con Rasterio para asegurar bandas 2, 3, 1 y escalando a 8-bit...
⏳ Cargando modelo YOLOv8 desde yolo_best.pt...
⏳ Iniciando inferencia por Tiles con array de píxeles...
Performing prediction on 209 slices.
✅ Inferencia completada. Palmas detectadas: 18922
💾 Imagen visual (PNG) guardada en: final_results/visual_map_centroids.png
💾 GeoJSON guardado en: final_results/detected_palms.geojson

🚀 PROCESO FINALIZADO CON ÉXITO


In [18]:
import os
import sys
import cv2
import torch
import torchvision
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from rasterio.windows import Window
# Importaciones específicas de Faster R-CNN (Tomadas de test1_2.py)
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models.resnet import ResNet101_Weights


# ==============================================================================
# ⚙️ CONFIGURATION
# ==============================================================================
IMAGE_PATH = "Sample-drone-data-copy.tif"
MODEL_PATH = "restnet101__exp63_best.pt" # <--- ¡Tu archivo .pt de Faster RCNN!
OUTPUT_DIR = "final_results_fasterrcnn"

# INFERENCE SETTINGS
SLICE_SIZE = 640         # Must match your training size
OVERLAP_RATIO = 0.2      # 20% overlap
CONFIDENCE = 0.25        # 25% minimum confidence
IOU_THRESHOLD = 0.5      # 50% overlap threshold (NMS)


# TILE_SIZE = 800
# OVERLAP = 100           # Píxeles de solapamiento
# CONFIDENCE = 0.65
# IOU_THRESHOLD = 0.3

# VISUALIZATION SETTINGS
DOT_SIZE = 10
DOT_COLOR = (0, 0, 255) # Red (BGR)
BG_COLOR = (0, 0, 0)


# ==============================================================================
# 1. FUNCIÓN DE CONSTRUCCIÓN DEL MODELO (tomada de test1_2.py)
# ==============================================================================
def get_fasterrcnn_model(num_classes, device, pretrained=False):
    """Crea la estructura del modelo ResNet101 + FPN y carga los pesos."""

    backbone_model = resnet_fpn_backbone(
        backbone_name='resnet101',
        weights=ResNet101_Weights.DEFAULT if pretrained else None,
        trainable_layers=5
    )

    model = FasterRCNN(backbone_model, num_classes=num_classes)

    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # Cargar tus pesos entrenados
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))

    model.to(device).eval()
    return model


# ==============================================================================
# 2. FUNCIÓN DE INFERENCIA PRINCIPAL
# ==============================================================================
def main():
    if not os.path.exists(IMAGE_PATH) or not os.path.exists(MODEL_PATH):
        sys.exit(f"❌ Error: Archivo TIFF o modelo .pt no encontrado.")

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print(f"✅ Usando dispositivo: {device}")

    # --------------------------------------
    # 1. CARGAR MODELO Y ABRIR RASTERIO
    # --------------------------------------
    detection_model = get_fasterrcnn_model(num_classes=2, device=device)
    all_detections = []

    with rasterio.open(IMAGE_PATH) as src:
        transform_base = src.transform
        crs = src.crs
        img_w, img_h = src.width, src.height

        # Leemos la imagen completa para visualización
        img_array_vis = src.read([2, 3, 1])

        # Conversión 16-bit a 8-bit para visualización
        max_val = np.max(img_array_vis)
        if max_val > 255:
            img_scaled = img_array_vis.astype(np.float32) * (255.0 / max_val)
            img_final_uint8 = img_scaled.astype(np.uint8)
        else:
            img_final_uint8 = img_array_vis.astype(np.uint8)

        img_cv_vis = np.transpose(img_final_uint8, (1, 2, 0))
        img_cv_vis = cv2.cvtColor(img_cv_vis, cv2.COLOR_RGB2BGR)

        # --------------------------------------
        # 2. LOOP DE TILING MANUAL (EL FIX)
        # --------------------------------------
        print(f"⏳ Iniciando inferencia en {img_w}x{img_h} con tiles de {TILE_SIZE} y {OVERLAP}px de solape...")

        step = TILE_SIZE - OVERLAP

        # Iteración manual sobre filas y columnas con el paso definido (step)
        for row in range(0, img_h, step):
            for col in range(0, img_w, step):

                # Definir la ventana de recorte: col_off/row_off indican el inicio
                # width/height se ajustan al límite de la imagen si es el último tile
                window = Window(
                    col_off=col,
                    row_off=row,
                    width=min(TILE_SIZE, img_w - col),
                    height=min(TILE_SIZE, img_h - row)
                )

                # Obtener la transformación específica para este tile
                transform = src.window_transform(window)

                # Leer el tile: (Canales, Alto, Ancho)
                tile_data = src.read(indexes=[2, 3, 1], window=window, boundless=False)

                # Aplicar la misma conversión 16-bit -> 8-bit al tile
                tile_max = np.max(tile_data)
                if tile_max > 255:
                    tile_scaled = tile_data.astype(np.float32) * (255.0 / tile_max)
                    tile_final_uint8 = tile_scaled.astype(np.uint8)
                else:
                    tile_final_uint8 = tile_data.astype(np.uint8)

                # Transponer a (H, W, C) y convertir a Tensor (C, H, W) [0, 1]
                tile_HWC = np.transpose(tile_final_uint8, (1, 2, 0))
                tile_tensor = torch.from_numpy(tile_HWC).float().permute(2, 0, 1) / 255.0

                # Agregar dimensión de batch y mover a GPU
                tile_tensor = tile_tensor.to(device).unsqueeze(0)

                # --------------------------------------
                # 3. INFERENCIA DEL TILE
                # --------------------------------------
                with torch.no_grad():
                    predictions = detection_model(tile_tensor)[0]

                # --------------------------------------
                # 4. PROCESAR Y GEO-REFERENCIAR
                # --------------------------------------
                scores = predictions['scores'].cpu().numpy()
                boxes = predictions['boxes'].cpu().numpy()

                # Filtrar por confianza (opcional, pero útil para reducir el trabajo de NMS)
                high_conf_indices = scores >= CONFIDENCE
                scores = scores[high_conf_indices]
                boxes = boxes[high_conf_indices]

                # Obtener el desplazamiento global del tile
                offset_x = window.col_off
                offset_y = window.row_off

                for score, box in zip(scores, boxes):
                    # 1. Convertir coordenadas del tile (px, py) a coordenadas GLOBALES (col, row)
                    box_global = [
                        box[0] + offset_x, box[1] + offset_y,
                        box[2] + offset_x, box[3] + offset_y
                    ]

                    # 2. Calcular Centroide Global
                    px = (box_global[0] + box_global[2]) / 2  # Columna
                    py = (box_global[1] + box_global[3]) / 2  # Fila

                    # 3. Geo-referenciar: (col, row) -> (x, y)
                    real_x, real_y = rasterio.transform.xy(transform_base, py, px, offset='center')

                    all_detections.append({
                        "score": score,
                        "boxes": box_global,
                        "pixel_x": px,
                        "pixel_y": py,
                        "real_x": real_x,
                        "real_y": real_y
                    })


        # --------------------------------------
        # 5. POST-PROCESAMIENTO (NMS GLOBAL)
        # --------------------------------------
        if not all_detections:
            print("⚠️ No se detectaron palmas. Proceso finalizado.")
            return

        print("⏳ Aplicando NMS Global para eliminar duplicados del solapamiento...")
        df_all = pd.DataFrame(all_detections)

        boxes_tensor = torch.as_tensor(df_all['boxes'].tolist(), dtype=torch.float32)
        scores_tensor = torch.as_tensor(df_all['score'].values, dtype=torch.float32)

        keep_indices = torchvision.ops.nms(boxes_tensor, scores_tensor, iou_threshold=IOU_THRESHOLD)

        final_df = df_all.iloc[keep_indices.cpu().numpy()]
        total_palms = len(final_df)
        print(f"✅ Detecciones finales después de NMS: {total_palms}")


        # --------------------------------------
        # 6. EXPORTAR GEOJSON y PNG VISUAL
        # --------------------------------------
        print("💾 Guardando resultados...")

        # Crear GeoDataFrame
        gdf = gpd.GeoDataFrame(
            final_df,
            geometry=[Point(xy) for xy in zip(final_df.real_x, final_df.real_y)],
            crs=crs
        )

        # 6.1 Guardar GeoJSON
        out_geojson = os.path.join(OUTPUT_DIR, "detected_palms_fasterrcnn.geojson")
        gdf.to_file(out_geojson, driver="GeoJSON")

        # 6.2 Guardar PNG Visual
        for index, row in final_df.iterrows():
            cv2.circle(img_cv_vis, (int(row['pixel_x']), int(row['pixel_y'])), DOT_SIZE, DOT_COLOR, -1)

        # Dashboard Label
        label_text = f" Palms: {total_palms} "
        cv2.rectangle(img_cv_vis, (0, 0), (350, 60), BG_COLOR, -1)
        cv2.putText(img_cv_vis, label_text, (10, 45),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

        out_png = os.path.join(OUTPUT_DIR, "mapa_deteccion_fasterrcnn.png")
        cv2.imwrite(out_png, img_cv_vis)

        print(f"\n🚀 PROCESO FINALIZADO CON ÉXITO.")
        print(f"Archivo GeoJSON: {out_geojson}")
        print(f"Imagen Visual:   {out_png}")


# --------------------------------------
# FIXED ENTRY POINT
# --------------------------------------
if __name__ == "__main__":
    main()

✅ Usando dispositivo: cuda
⏳ Iniciando inferencia en 9424x5603 con tiles de 800 y 100px de solape...
⏳ Aplicando NMS Global para eliminar duplicados del solapamiento...
✅ Detecciones finales después de NMS: 8083
💾 Guardando resultados...

🚀 PROCESO FINALIZADO CON ÉXITO.
Archivo GeoJSON: final_results_fasterrcnn/detected_palms_fasterrcnn.geojson
Imagen Visual:   final_results_fasterrcnn/mapa_deteccion_fasterrcnn.png
